# KSI monitoring: úlohy

V tomto souboru se nachází hlavní tabulka statistik týkajících se úloh.

In [4]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import sys
sys.path.append('..')

import matplotlib.pyplot as plt
from collections import OrderedDict, namedtuple
from sqlalchemy import func, distinct, text, and_
import pandas as pd
from IPython.display import display, HTML, Markdown
import seaborn as sns
import numpy as np

import util
from util.year import year as current_year
from db import session
import model
from datetime import datetime

pd.options.display.float_format = '{:.2f}'.format
plt.rcParams['figure.figsize'] = [8, 6]
print(datetime.now())

2019-08-02 19:20:55.498887


In [6]:
evaluations = session.query(
    model.Task,
    func.count(distinct(model.User.id)).label('evals_count'),
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user).\
    filter(model.User.role == 'participant')

evaluations_per_task = evaluations.\
    group_by(model.Task).order_by(model.Wave.id, model.Task.id)

evaluations_per_task_d = {
    task: evals_count
    for (task, evals_count) in evaluations_per_task.all()
}

In [7]:
successful_evaluations = evaluations_per_task.\
    filter(model.Evaluation.ok == True)

successful_evaluations_d = {
    task: evals_count
    for (task, evals_count) in successful_evaluations.all()
}

In [8]:
per_module = session.query(
    model.Evaluation.user.label('user_id'),
    func.count(model.Evaluation.id).label('eval_count'),
).\
    join(model.Evaluation.r_module).join(model.Module.r_task).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Evaluation.user,
             model.Evaluation.module).subquery()

EVAL_LIMITS = [10, 30, 50]
problematic_tasks = {
    limit: {
        task: evals_count
        for (task, evals_count) in (
            evaluations.
            filter(model.Evaluation.ok == False).
            join(per_module, per_module.c.user_id == model.User.id).\
            group_by(model.Task).\
            filter(per_module.c.eval_count > limit).all()
        )
    }
    for limit in EVAL_LIMITS
}

In [9]:
posts_count = session.query(
    model.Task,
    func.count(model.Post.id)
).\
    join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    join(model.Task.discussion_posts).\
    group_by(model.Task).\
    all()

posts_count_dict = {
    task: count
    for (task, count) in posts_count
}

In [10]:
all_waves = session.query(model.Wave).\
    filter(model.Wave.year == current_year.id).\
    order_by(model.Wave.index).all()

large_tasks = util.task.large_tasks().all()

In [11]:
no_evaluations_per_task_and_user = session.query(
    model.Task,
    model.User,
    func.count(model.Evaluation.id),
).\
    join(model.Task.r_wave).filter(model.Wave.year == current_year.id).\
    join(model.Task.modules).\
    join(model.Module.evaluations).\
    join(model.Evaluation.r_user).\
    filter(model.User.role == 'participant').\
    group_by(model.Task, model.User).\
    all()

In [12]:
no_evaluations_dict = {}
for task, user, count in no_evaluations_per_task_and_user:
    if task in no_evaluations_dict:
        no_evaluations_dict[task].append(count)
    else:    
        no_evaluations_dict[task] = [count]
        
no_evaluations_avg = {
    task: np.average(counts)
    for task, counts in no_evaluations_dict.items()
}

no_evaluations_median = {
    task: np.median(counts)
    for task, counts in no_evaluations_dict.items()
}

In [13]:
def show_wave_stats(wave, max_evals_count):
    tasks = [
        (
            task,
            evaluations_per_task_d[task],
            successful_evaluations_d[task],
            successful_evaluations_d[task] / evaluations_per_task_d[task],
            evaluations_per_task_d[task]-successful_evaluations_d[task],
            problematic_tasks[10][task] if task in problematic_tasks[10] else 0,
            problematic_tasks[30][task] if task in problematic_tasks[30] else 0,
            problematic_tasks[50][task] if task in problematic_tasks[50] else 0,
            posts_count_dict[task] if task in posts_count_dict else 0,
            no_evaluations_avg[task] if task in no_evaluations_avg else '-',
            no_evaluations_median[task] if task in no_evaluations_median else '-',
            sorted(no_evaluations_dict[task])[-5:] if task in no_evaluations_dict else '-',
        )
        for task in evaluations_per_task_d
        if task.wave == wave.id
    ]
    tasks.sort(key=lambda x: (x[0] in large_tasks, x[0].id))

    df = pd.DataFrame(tasks, columns=[
        'Task',
        'All Evaluations',
        'Successful Evaluations',
        'Successful/All ratio',
        'N.O. Users failing on task now',
        'N.O. Users with more than 10 unsucc. evaluations',
        'N.O. Users with more than 30 unsucc. evaluations',
        'N.O. Users with more than 50 unsucc. evaluations',
        'Discussion Posts Count',
        'Average N.O. Evaluations Per User',
        'Median N.O. Evaluations Per User',
        'Some Maximum Evaluations Counts',
    ]).set_index('Task')
    
    s = df.style
    
    s.background_gradient(subset=[
        'N.O. Users failing on task now',
        'N.O. Users with more than 10 unsucc. evaluations',
        'N.O. Users with more than 30 unsucc. evaluations',
        'N.O. Users with more than 50 unsucc. evaluations',
        'Average N.O. Evaluations Per User',
        'Median N.O. Evaluations Per User',        
    ], cmap=sns.light_palette("red", as_cmap=True))
    
    s.background_gradient(
        subset=['Successful/All ratio'],
        cmap=sns.light_palette("green", as_cmap=True)
    )
    
    s.background_gradient(
        subset=['Discussion Posts Count'],
        cmap=sns.light_palette("orange", as_cmap=True)
    )
    
    s.bar(subset=['All Evaluations'], color='#5fd65f', vmin=0, vmax=max_evals_count)
    
    s.format({
        'Successful/All ratio': '{:,.1%}'.format,
        'Average N.O. Evaluations Per User': '{:.2f}'.format,
    })
    
    display(Markdown('## {name}'.format(name=wave.caption)))
    display(s)

max_evals_count = max(evaluations_per_task_d.values(), default=10)
    
for wave in all_waves:
    show_wave_stats(wave, max_evals_count)

## Nultá vlna

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations,Discussion Posts Count,Average N.O. Evaluations Per User,Median N.O. Evaluations Per User,Some Maximum Evaluations Counts
Task,,,,,,,,,,,


## Stromy a algoritmy + vyvažování

,All Evaluations,Successful Evaluations,Successful/All ratio,N.O. Users failing on task now,N.O. Users with more than 10 unsucc. evaluations,N.O. Users with more than 30 unsucc. evaluations,N.O. Users with more than 50 unsucc. evaluations,Discussion Posts Count,Average N.O. Evaluations Per User,Median N.O. Evaluations Per User,Some Maximum Evaluations Counts
Task,,,,,,,,,,,


## Statistika velkých úloh

In [14]:
per_module = session.query(
    model.Wave,
    model.Task,
    model.Evaluation.user.label('user_id'),
    model.Evaluation.module.label('module_id'),
    func.max(model.Evaluation.points).label('points'),
    model.Module.max_points.label('max_points'),
).\
    join(model.Evaluation.r_module).\
    filter(model.Module.type == model.ModuleType.GENERAL).\
    join(model.Module.r_task).join(model.Task.r_wave).\
    filter(model.Wave.year == current_year.id).\
    group_by(model.Task, model.Evaluation.user, model.Evaluation.id).\
    order_by(model.Wave.id)

norm_points = [
    pm.points / pm.max_points
    for pm in per_module
]

df = pd.DataFrame(per_module.all())
if not df.empty:
    df['norm_points'] = norm_points
    grouped = df.groupby('Task')

    per_task = pd.DataFrame(OrderedDict((
        ('Wave', grouped.Wave.first()),
        ('Solved count', grouped.user_id.count()),
        ('Max Points', grouped.max_points.first()),
        ('Points Average', grouped.points.mean()),
        ('Points Median', grouped.points.median()),    
        ('Points Average Normalized', grouped.norm_points.mean()),
        ('Points Median Normalized', grouped.norm_points.median()),
    )))
    s = per_task.style
    s.format({
        'Points Average': '{:.2f}'.format,
        'Points Average Normalized': '{:,.1%}'.format,
        'Points Median Normalized': '{:,.1%}'.format,
    })

    s.background_gradient(subset=[
        'Solved count',
        'Points Average Normalized',
        'Points Median Normalized',
    ], cmap=sns.light_palette("green", as_cmap=True))

    s.background_gradient(subset=[
        'Points Median',
        'Points Average',
    ], cmap=sns.light_palette("blue", as_cmap=True))

    display(s)

## Kolik řešitelů získalo kolik bodů za velké úlohy

In [16]:
if not df.empty:
    gained_points = pd.DataFrame(OrderedDict((
        ('[9, inf)', df[df['points'] >= 9].groupby('Task').points.count()),    
        ('[8–9)', df[(df['points'] >= 8) & (df['points'] < 9)].groupby('Task').points.count()),    
        ('[6–8)', df[(df['points'] >= 6) & (df['points'] < 8)].groupby('Task').points.count()),
        ('[4–6)', df[(df['points'] >= 4) & (df['points'] < 6)].groupby('Task').points.count()),

        ('[0–4)', df[(df['points'] >= 0) & (df['points'] < 4)].groupby('Task').points.count()),
    )))
    gained_points.plot.bar(stacked=True);

## Předchozí tabulka normalizovaná maximálním počtem bodů za úlohu

In [18]:
if not df.empty:
    gained_points_norm = pd.DataFrame(OrderedDict((
        ('>90 %', df[df['norm_points'] >= .9].groupby('Task').norm_points.count()),    
        ('[80 % – 90 %)', df[(df['norm_points'] >= .8) & (df['norm_points'] < .9)].groupby('Task').norm_points.count()),
        ('[60 % – 80 %)', df[(df['norm_points'] >= .6) & (df['norm_points'] < .8)].groupby('Task').norm_points.count()),
        ('[40 % – 60 %)', df[(df['norm_points'] >= .4) & (df['norm_points'] < .6)].groupby('Task').norm_points.count()),
        ('[0 % – 40 %)', df[(df['norm_points'] >= 0) & (df['norm_points'] < .4)].groupby('Task').norm_points.count()),
    )))
    gained_points_norm.plot.bar(stacked=True);